In [54]:
!pip install openai
!pip install googletrans==4.0.0-rc1
!pip install Faker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 20.3 MB/s eta 0:00:00


In [2]:

import os

def mk_File(file_name, file_content, path):
    # 파일 경로와 이름을 지정합니다.
    file_path = "/Users/bagjeong-yong/desktop/hk2/gpt_h"
    
    # 파일이 이미 존재하면 삭제합니다.
    if os.path.exists(os.path.join(file_path, file_name)):
        os.remove(os.path.join(file_path, file_name))

    # 파일 디렉토리가 존재하지 않으면 생성합니다.
    if not os.path.exists(file_path):
        os.makedirs(file_path)

    # 파일 객체를 생성합니다.
    with open(os.path.join(file_path, file_name), "w") as f:
        # 파일에 내용을 작성합니다.
        f.write(file_content)

    print(f"{os.path.join(file_path, file_name)} 파일이 생성되었습니다.")
    

In [3]:
import os
import openai
import json

def code_file():
    ti, lang = input("파일 이름 with 파일 유형(.py, .c, .cpp ...etc) \n").split(".")
    content = input("만드려는 코드 내용(30자 이내) : ")
    openai.api_key = ""
    
    response = openai.Completion.create(
        model="text-davinci-003",
        prompt=content + " as ." + lang,
        temperature=0,
        max_tokens=100,
        top_p=1,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )

    fc = response.choices[0].text.strip()

    path = "C:/Users/sunbum/Desktop/공모전/temp_files"
    mk_File(ti +"."+ lang, fc, path)
    

In [4]:
import googletrans
# 한글을 판별합니다.
def is_korean(word):
    for char in word:
        if '가' <= char <= '힣':
            return True
    return False


def e_translate(k_word):
  translator = googletrans.Translator()
  # 라이브러리를 사용하여 번역합니다.
  if is_korean(k_word):
    outStr = translator.translate( k_word, dest = 'en', src = 'auto')
    return outStr.text
  else:
    return k_word

def k_translate(e_word):
  translator = googletrans.Translator()
  # 라이브러리를 사용하여 번역합니다.
  outStr = translator.translate( e_word, dest = 'ko', src = 'auto')
    
  return outStr.text


In [5]:

# 필요한 모듈을 import 합니다.
import os
import openai
import json

def word_file():

  openai.api_key = ""

  # 파일 내용을 Openai를 이용해 생성합니다.
  ti, lang = input("파일 이름 with 파일 유형(.docx, .hwp, ...etc) \n").split(".")
  role = e_translate(input("주제 분야 (과학, 인문 ..etc): "))
  title = e_translate(input("파일 목적( ex 과학혁명의 구조 보고서): "))
  topic = e_translate(input("들어가야할 내용 (동기, 줄거리 , 결론 .. etc): "))

  # 모델 엔진을 지정합니다.
  model = "text-davinci-003"
  # 생성할 최대 토큰 수를 지정합니다.
  max_tokens = 1024
  # 생성된 텍스트의 창의성과 예측 가능성을 조절합니다.
  temperature = 0.5

  prompt = f"I'm in in the field of {role}. I'm going to write {title}. Write a report using according to {topic}"

  response = openai.Completion.create(
      engine=model,
      prompt=prompt,
      temperature=temperature,
      max_tokens=max_tokens,
      n = 1,
      stop = None
  )

  fc = k_translate(response.choices[0].text)
  print(fc)
  path = "/Users/bagjeong-yong/desktop/hk2/gpt_h"
  mk_File(ti +"."+ lang, fc, path)


In [147]:
from faker import Faker
import random
def generate_data(cols, n):

    fake = Faker('ko_KR')

    data = []
    for i in range(n):
        row = []
        for j in range(len(cols)):
            if str(cols[j]) == '1':
                row.append(fake.name())
            elif str(cols[j])== '2':
                row.append(random.choice(['Active', 'Inactive']))
            elif str(cols[j]) == '3':
                row.append(fake.date_between(start_date='-1y', end_date='today')) #replace with random date
            elif str(cols[j]) == '4':
                row.append(fake.date_between(start_date='+1d', end_date='+1y'))
            elif str(cols[j])== '5':
                row.append('customer' + str(i) + '@example.com') #replace with random email
            elif str(cols[j]) == '6':
                row.append(random.randint(20, 60))
            elif str(cols[j]) == '7':
                row.append(random.choice(['A', 'B', 'C', 'D']))
            elif str(cols[j]) == '8':
                row.append(random.randint(10000, 50000))
            elif str(cols[j]) == '9':
                row.append(fake.phone_number())
            elif str(cols[j]) == '10':
                row.append(fake.address())
            elif str(cols[j]) == '11':
                row.append(i)
            else:
                row.append("null")
        data.append(row)
    return data

import string
def col_num_to_str(col_num):
    if col_num <= 0:
        raise ValueError('열 번호는 1 이상이어야 합니다.')
    col_str = ''
    while col_num > 0:
        remainder = (col_num - 1) % 26
        col_str = string.ascii_uppercase[remainder] + col_str
        col_num = (col_num - 1) // 26
    return col_str


# 필요한 모듈을 import 합니다.
import os
import openai
import json
import ast
import pandas as pd
import openpyxl 
from openpyxl import load_workbook

def excel_file():

  openai.api_key = ""

  # 파일 내용을 Openai를 이용해 생성합니다.
  ti, lang = input("엑셀 제목 with 파일 유형(xlsx, ...etc) \n").split(".")
  role = e_translate(input("주제 분야 (고객관리 ..etc): "))
  n_row = str(input("행 개수 : "))
  n_column = str(input("열 개수 : "))

  # 모델 엔진을 지정합니다.
  model = "text-davinci-003"
  # 생성할 최대 토큰 수를 지정합니다.
  max_tokens = 500
  # 생성된 텍스트의 창의성과 예측 가능성을 조절합니다.
  temperature = 0.5

  
  prompt = f"You're a consultant. I'm trying to make a member management excel file with a size of {n_row} about {role}. Give me an example by wrapping up the contents of the line with \\\ "
  
  response = openai.Completion.create(
      engine=model,
      prompt=prompt,
      temperature=temperature,
      max_tokens=max_tokens,
      n = 1,
      stop = None
  )

  fc = response.choices[0].text.split("\\")
  new_list = [element.strip('').replace(":",'').replace(" ",'').replace("\n",'') for element in fc if element.strip()]
  print(new_list)

  ask_list = ["?" for i in range(int(n_row))]

  prompt = f"어떠한 행이 주어지면 행의 각 항목이 1(이름 - 3글자이하 문자반환), 2(시작날짜 - 끝날짜보다 작은 날짜 반환), 3(끝 - 시작날짜보다 큰 날짜 반환), 4(이메일 - 문자@ex.com 반환), 5(나이 - 20~50 사이의 수 반환), 6(활성상태 - 2가지 상태를 반환), 7(성적 - A~D 중 임의의 알파벳 반환) 8(가격- 10000~50000이하의 수 반환) 9(핸드폰 번호 - ###-####-### 반환),10(지역- 지역을 표시하는 문자열 반환) ,11(고객id - 고객별 고유번호 생성 ), 12(해당사항없음) 중 가장 연관성있는 번호를 찾고 그에따라 데이터를 생성하려고 해. {new_list} 에 각 항목들은 어떤 번호에 가장 가까울까? 답의 형식은 \ {ask_list} \ 이다.."
  
  response = openai.Completion.create(
      engine=model,
      prompt=prompt,
      temperature=temperature,
      max_tokens=max_tokens,
      n = 1,
      stop = None
  )

 
  result_list = ast.literal_eval(response.choices[0].text)
  print(result_list)
  matrix = generate_data(result_list,int(n_column))
  print(matrix[0])
  df = pd.DataFrame(matrix , columns=new_list)
  print(df)
  df.to_excel(ti +"."+ lang)

  while True:
    print("기능 : < 정렬, 합산, 평균, 최소, 최대 > 중 선택")
    func = input("원하는 기능입력( 형식 : 최대, 평균..) 추가를 원하지 않을 경우 q입력")
    if func == 'q':
      break
    t_col = col_num_to_str(int(input(f"{func}값을 구하고 싶은 열 입력 (형식 : 2..)")))
    n_column = int(n_column)
    if func == '정렬':
        value = f'=SORT({t_col}1:{t_col}{n_column})'
    elif func == '합산':
        value = f'=SUM({t_col}1:{t_col}{n_column})'
    elif func == '평균':
        value = f'=AVERAGE({t_col}1:{t_col}{n_column})'
    elif func == '최소':
        value = f'=MIN({t_col}1:{t_col}{n_column})'
    elif func == '최대':
        value = f'=MAX({t_col}1:{t_col}{n_column})'
    else:
        print("유효하지 않은 값")
        break

    wb = load_workbook(ti +"."+ lang)
    ws = wb.active
    
    ws[f"{t_col}{n_column+1}"] = value
    wb.save(ti +"."+ lang)





In [150]:
import os

flag_on = True
while flag_on:
    print("생성할 파일 유형을 선택해주세요\n 1. PPT\n 2. Excel\n 3. Word \n 4. Source File \n 0. exit ")
    big_cate = int(input())
    if big_cate == 1:
        # ppt class
        a = 0
    elif big_cate == 2:
        # 엑셀 class
        excel_file()
    elif big_cate == 3:
        # 워드 class
        word_file()
    elif big_cate == 4:
        # 코드 소스파일 class
        code_file()
    elif big_cate == 0:
        break
    else:
        print("잘못된 입력")
        continue


생성할 파일 유형을 선택해주세요
 1. PPT
 2. Excel
 3. Word 
 4. Source File 
 0. exit 
2
엑셀 제목 with 파일 유형(xlsx, ...etc) 
fas.xlsx
주제 분야 (고객관리 ..etc): 헬스장 회원관리
행 개수 : 6
열 개수 : 20
['Column1MemberID', 'Column2Name', 'Column3DateofBirth', 'Column4ContactNumber', 'Column5MembershipType', 'Column6PaymentStatus']
['11', '2', '3', '9', '6', '7']
[0, 'Inactive', datetime.date(2022, 10, 23), '017-357-3351', 41, 'B']
    Column1MemberID Column2Name Column3DateofBirth Column4ContactNumber  \
0                 0    Inactive         2022-10-23         017-357-3351   
1                 1      Active         2023-01-20         062-938-6814   
2                 2      Active         2022-05-13         018-745-7954   
3                 3      Active         2023-04-16         02-9687-6346   
4                 4      Active         2022-05-14         032-895-9573   
5                 5      Active         2022-12-03         019-456-5272   
6                 6      Active         2022-08-23         044-545-6478   
7   

ValueError: ignored